In [300]:
#Handle Data
import pandas as pd
import numpy as np

#Handle Images
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#counter
from collections import Counter

#show plots and images
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score


In [346]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),          # Resize all images to 128x128
    transforms.ToTensor(),                  # Convert images to PyTorch tensors
    transforms.Normalize([0.5], [0.5])      # Normalize (adjust if 3-channel RGB)
])

In [347]:
dataset = datasets.ImageFolder('train', transform=transform)

# Split into DataLoader for batching
dataloader = DataLoader(dataset, batch_size=1000, shuffle=True)

In [348]:
# One batch of data
images, labels = next(iter(dataloader))

print(images.shape)  # (batch_size, channels, height, width)
print(labels)        # 0 for healthy, 1 for infected (automatically labeled based on folder names)


torch.Size([1000, 3, 128, 128])
tensor([ 4,  5,  1,  3,  3,  3,  4,  2,  1,  3,  4,  2,  2,  9, 10,  2,  2,  9,
         8,  9,  1,  3,  4,  8,  4,  7,  8,  1,  8,  4,  9,  2,  9,  9,  2,  9,
         8,  0,  8,  4,  3,  6,  1,  2,  7,  3,  0,  1,  5,  7,  2,  4,  5,  0,
         2,  5,  9,  4,  2,  9,  3,  9,  2,  5,  5,  8,  0,  6,  8,  7,  5,  6,
         6,  6,  1,  0,  4,  2,  9,  9,  8,  2,  2,  3,  0,  2,  9,  9, 10,  7,
         6,  6,  2,  3,  4,  4,  6,  2,  3,  1,  0,  5,  2,  8,  6,  5,  4,  1,
         5,  3,  1,  1,  4,  4,  6,  8,  3,  7,  6,  9,  5,  3,  3,  3,  7,  8,
         0,  4,  2,  8, 10,  3,  4,  0,  3,  7,  9,  9,  9,  2,  3,  6,  7,  8,
         8,  1,  8,  1,  1,  5,  4,  3,  0,  0,  2,  3, 10,  5,  5,  7,  0,  5,
         0,  9,  3,  3,  9,  5,  3,  1,  0,  5,  0,  1,  6,  2,  4,  0,  4,  1,
         1,  4,  9,  9,  2,  5,  7,  9,  2,  9,  0,  4,  9,  6,  9,  9,  8, 10,
         7,  4,  4,  1,  8,  3,  1,  0,  6,  8,  5,  7,  8,  8,  4,  0,  1,  3,
        

In [349]:
flattened = images.view(images.size(0), -1)
print(flattened.shape)  # (batch_size, 128*128*3) for RGB images
print(labels.shape)

torch.Size([1000, 49152])
torch.Size([1000])


In [350]:
from sklearn.decomposition import PCA

# Flatten images
X_flat = flattened

# Reduce dimensions (e.g., 50)
pca = PCA(n_components=17, whiten = True)
X_reduced = pca.fit_transform(X_flat)
X_reduced.shape

(1000, 17)

In [351]:
X = np.array(X_reduced)
y = np.array(labels)
X

array([[ 0.43623879,  2.10039317, -0.99784053, ..., -0.66264537,
         1.66167612,  0.21972036],
       [ 0.24372807, -1.00077355,  0.71397607, ...,  0.22526593,
        -0.52581742, -0.22456132],
       [ 0.58909021, -1.05038007, -0.85364413, ..., -0.3145926 ,
         0.52254548, -0.22123797],
       ...,
       [-0.04807696, -0.43815069, -1.16074321, ..., -0.62964657,
        -0.0753472 , -0.17375244],
       [-0.1521742 ,  1.22870965,  0.76051026, ...,  0.60212489,
         1.81488842,  1.32339071],
       [-0.15753691, -0.76397259, -0.78540857, ..., -0.12687661,
        -0.51449744, -0.12918221]])

In [352]:
def standardization(X):
    X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

    return X

X = standardization(X)
X

array([[ 0.43645708,  2.10144416, -0.99833982, ..., -0.66297694,
         1.66250758,  0.2198303 ],
       [ 0.24385002, -1.00127432,  0.71433333, ...,  0.22537865,
        -0.52608053, -0.22467368],
       [ 0.58938497, -1.05090566, -0.85407127, ..., -0.31475002,
         0.52280695, -0.22134867],
       ...,
       [-0.04810102, -0.43836993, -1.16132401, ..., -0.62996163,
        -0.0753849 , -0.17383939],
       [-0.15225034,  1.22932447,  0.7608908 , ...,  0.60242617,
         1.81579655,  1.3240529 ],
       [-0.15761574, -0.76435486, -0.78580157, ..., -0.1269401 ,
        -0.51475488, -0.12924685]])

In [353]:
class_names = dataset.classes
class_names

['Bacterial_spot',
 'Early_blight',
 'Late_blight',
 'Leaf_Mold',
 'Septoria_leaf_spot',
 'Spider_mites Two-spotted_spider_mite',
 'Target_Spot',
 'Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato_mosaic_virus',
 'healthy',
 'powdery_mildew']

In [354]:
def standardization(X):
        X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    
        return X

class KNNClassifier:
    def __init__(self, k = 3):
        self.k = k

    def fit(self, train_x, train_y):
        self.train_x = standardization(np.array(train_x))
        self.train_y = np.array(train_y)

    def predict(self, test_x):
        self.test_x = standardization(np.array(test_x))

        predictions = []
        for x in self.test_x:
            dist = np.linalg.norm(self.train_x - np.array(x), axis = 1)
            min_indices = dist.argsort()[:self.k]
            min_ys = self.train_y[min_indices]
            most_common = Counter(min_ys).most_common()[0][0]
            predictions.append(most_common)
        return predictions
            

In [355]:
Counter([1,2,4,5,3,1,3,2,3]).most_common()[0][0]

3

In [356]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.2, random_state=42)

In [357]:
model = KNNClassifier(k=6)
model.fit(X_train, y_train)
result = model.predict(X_test)

In [358]:
y_test

array([ 9,  4,  9,  5,  3, 10,  9,  0,  0,  9,  1,  4,  8,  1,  9,  3,  3,
        5,  2, 10,  4,  6,  2,  7,  8,  8,  5,  3,  6,  2,  0,  2,  2,  9,
        7,  7,  1, 10,  3,  9,  0,  6,  0,  5,  5,  3,  8,  0,  9,  7,  0,
        4,  0,  1,  8,  9,  1,  4,  0,  0,  5,  2,  7,  7,  6,  3,  2,  8,
        5,  7,  1,  2,  9,  0,  4, 10,  1,  4,  4,  8,  4,  0,  4,  6, 10,
        9,  9,  7,  7,  0,  4,  1,  2,  6,  0,  6,  5, 10,  5, 10,  6,  0,
        4,  6, 10,  6, 10,  6,  3,  2,  2,  2,  5,  9,  9,  5,  2,  4,  9,
        0,  8,  3,  0, 10,  4,  4,  5,  0,  7,  4,  1,  3,  1,  5,  9,  5,
        5,  3,  7,  6,  7, 10,  7,  0,  8,  8,  0,  3,  8,  7,  0,  5,  6,
        3,  9,  2,  5,  2,  3,  9,  6,  0,  1,  5,  6, 10,  1,  2,  4,  9,
        2,  3,  1,  8,  0,  6,  4,  5,  5,  9,  4,  2,  9,  1,  6,  9,  5,
        5,  7,  7,  9,  4,  2,  4,  0,  6,  6,  5,  4,  9], dtype=int64)

In [359]:
result

[9,
 6,
 10,
 5,
 3,
 5,
 9,
 0,
 0,
 9,
 3,
 4,
 0,
 2,
 9,
 1,
 4,
 8,
 2,
 7,
 7,
 9,
 9,
 7,
 4,
 8,
 0,
 3,
 8,
 0,
 0,
 1,
 2,
 9,
 0,
 0,
 6,
 3,
 3,
 7,
 0,
 4,
 2,
 5,
 5,
 3,
 8,
 0,
 8,
 7,
 9,
 4,
 0,
 0,
 8,
 0,
 9,
 4,
 9,
 0,
 6,
 9,
 7,
 7,
 8,
 2,
 2,
 8,
 9,
 0,
 0,
 9,
 9,
 0,
 6,
 0,
 0,
 3,
 0,
 8,
 7,
 0,
 4,
 6,
 4,
 8,
 3,
 7,
 7,
 7,
 4,
 6,
 3,
 5,
 2,
 0,
 5,
 2,
 5,
 9,
 6,
 2,
 4,
 9,
 9,
 5,
 3,
 6,
 3,
 3,
 2,
 0,
 0,
 9,
 2,
 2,
 0,
 4,
 9,
 2,
 3,
 3,
 6,
 2,
 4,
 9,
 5,
 9,
 1,
 1,
 8,
 5,
 4,
 5,
 10,
 5,
 6,
 3,
 7,
 6,
 7,
 2,
 7,
 0,
 7,
 4,
 0,
 2,
 2,
 7,
 2,
 9,
 5,
 8,
 1,
 4,
 5,
 2,
 8,
 9,
 5,
 3,
 1,
 0,
 0,
 8,
 1,
 2,
 2,
 9,
 2,
 3,
 0,
 2,
 2,
 0,
 2,
 6,
 5,
 9,
 4,
 2,
 9,
 0,
 6,
 9,
 5,
 5,
 7,
 7,
 9,
 4,
 0,
 3,
 0,
 0,
 9,
 1,
 2,
 1]

In [360]:
p = 0
for i in range(len(y_test)):
    if y_test[i] == result[i]:
        p = p + 1

p/len(y_test)

0.44

In [361]:
for k in range(1, 30):
    model = KNNClassifier(k=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"k = {k}, accuracy = {acc:.4f}")

k = 1, accuracy = 0.3950
k = 2, accuracy = 0.3950
k = 3, accuracy = 0.4400
k = 4, accuracy = 0.4400
k = 5, accuracy = 0.4400
k = 6, accuracy = 0.4400
k = 7, accuracy = 0.4700
k = 8, accuracy = 0.4700
k = 9, accuracy = 0.4650
k = 10, accuracy = 0.4700
k = 11, accuracy = 0.4900
k = 12, accuracy = 0.4850
k = 13, accuracy = 0.4850
k = 14, accuracy = 0.4850
k = 15, accuracy = 0.4800
k = 16, accuracy = 0.4900
k = 17, accuracy = 0.4650
k = 18, accuracy = 0.4550
k = 19, accuracy = 0.4650
k = 20, accuracy = 0.4400
k = 21, accuracy = 0.4250
k = 22, accuracy = 0.4350
k = 23, accuracy = 0.4350
k = 24, accuracy = 0.4350
k = 25, accuracy = 0.4350
k = 26, accuracy = 0.4300
k = 27, accuracy = 0.4100
k = 28, accuracy = 0.4100
k = 29, accuracy = 0.4050
